In [1]:
HYDRA_FULL_ERROR=1
import hydra
from typing import Optional
from collections.abc import Callable
from omegaconf import OmegaConf, DictConfig

from hydra import initialize, initialize_config_module, initialize_config_dir, compose

from hydra.utils import instantiate
from torchmetrics import MetricCollection

        
def setup_config(cfg: DictConfig):
    '''
    # TODO default_config.yaml에서 save_dir: ${hydra:runtime.cwd}/logs/ resolve 할 때 error 생김
    # TODO resolve 하기 전에 save_dir key remove 해주는 코드 추가로 필요함
    '''
    OmegaConf.set_struct(cfg, False)
    OmegaConf.resolve(cfg)
    OmegaConf.set_struct(cfg, True)

In [2]:
# *  config setup  * #
CONFIG_PATH = '/usr/src/CV_For_Autonomous_Driving/config'

with initialize_config_dir(config_dir=CONFIG_PATH):
    cfg = compose(config_name='default_config.yaml')

setup_config(cfg)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default_config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [15]:
metrics = {k: v for k, v in instantiate(cfg.metrics).items()}
metrics_collections = MetricCollection(metrics)

/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (IoUMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [16]:
for k, v in metrics_collections.items():
    print(k, v)

lane_iou IoUMetric()
road_iou IoUMetric()
vehicle_iou IoUMetric()


In [17]:

prefix = 'train'

# for _, metrics_collection in metrics_collections.items():
#     metrics = metrics_collection.compute()
#     for k, v in metrics.items():
#         print(f'{prefix}/metrics/{k}', v)

metrics = metrics_collections.compute()
print(metrics)
for key, value in metrics.items():
    if isinstance(value, dict):
        for subkey, val in value.items():
            print(f'{prefix}/metrics/{key}{subkey}', val)
    else:
        print(f'{prefix}/metrics/{key}', value)

{'lane_iou': 0.0, 'road_iou': 0.0, 'vehicle_iou': 0.0}
train/metrics/lane_iou 0.0
train/metrics/road_iou 0.0
train/metrics/vehicle_iou 0.0


In [ ]:
train/metrics/@0.40 0.0
train/metrics/@0.50 0.0

train/metrics/road_iou 0.0

train/metrics/road_iou IoUMetric()
train/metrics/lane_iou IoUMetric()
train/metrics/vehicle_iou IoUMetric()

In [6]:
from model_module.losses import MultipleLoss
loss_func = MultipleLoss(instantiate(cfg.loss))

In [7]:
for k, v in loss_func.items():
    print(k, v)

road BinarySegmentationLoss()
lane BinarySegmentationLoss()
vehicle BinarySegmentationLoss()
center CenterLoss()
